# LCA with Brightway2.5 - Ecoinvent

This notebook was designed to conduct LCAs using the brightway2.5 framework. I will add content in the future and make the code run more efficiently. For now, everything works within its limits and newcomers should have an easy entry to LCA using brightway.  

If you need to read up on literature, I provided some links about the tools, you'll be working with:  

1. [Python](https://www.python.org/)
2. [Brightway2.5](https://docs.brightway.dev/en/legacy/)
3. [Ecoinvent 3.11](https://ecoquery.ecoinvent.org/3.11/cutoff)
4. [Premise](https://premise.readthedocs.io/en/latest/)

Before you start, check the kernel on the top right. It should be "Python (env_bw25)" or whatever you named it in your virtual environment. If you are missing some functionality, check the brightway [cheat sheet](https://docs.brightway.dev/en/latest/content/cheatsheet/index.html).  

Enough blib blab, let's get started! 👾

### Step 0: Preparation

In [ ]:
# Start by importing the brightway packages and some addons to calculate the results.
# There is no magic happening yet so don't get all excited.

# Brightway packages.
import bw2data as bd
import bw2io as bi
import bw2calc as bc

# Packages for calculation and stuff.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# System & authentication packages.
import os
from dotenv import load_dotenv

# Ecoinvent packages.
from ecoinvent_interface import Settings, EcoinventRelease, ReleaseType

# Extra package to create codes for your activities.
import hashlib

# Packages for prospective LCA - currently only possbile for ecoinvent3.10 and lower
from premise import *

# Brightway version check, in case an error occurs.
print("bw2data version", bd.__version__)
print("bw2io version", bi.__version__)
print("bw2calc version", bc.__version__)

### Step 1: Project setup

Brightway is organized in projects to separate the individual settings of your LCA. Project management functions are included and self-explanatory.  

Continue by setting up your project.

In [ ]:
# Checking the projects never hurts and gives you an overview of all available projects.
bd.projects

# Optional: delete depricated projects; delete_dir=True deletes the project, delete_dir=False hides the project
# bd.projects.delete_project(name='<your_project>', delete_dir=True)

In [ ]:
# Choose your project: activate an existing project or create and activate a new one.
bd.projects.set_current("premise_test")
bd.projects

### Step 2: Import Ecoinvent as background 

Before you can start importing the background database, you need to store your credentials. Otherwise, you won't have access to the data. For convenience, you can use an .env file where you store the user and passcode information locally on your machine.  
1. Create an .env file next to this notebook.
2. Get your Ecoinvent credentials and save them as variables in the .env like so:

> USER="YOUR_USERNAME"  
> PASSCODE="YOUR_PASSWORD"

The .env keeps your credentials out of this notebook.

In [ ]:
# Check the current databases in your active project.
bd.databases

In [ ]:
# Authenticate to Ecoinvent to check available databases.
load_dotenv()
auth = Settings(username=os.getenv('USER'), password=os.getenv('PASSCODE'))
release = EcoinventRelease(auth)
release.list_versions()

In [ ]:
# Insert the db version you want to work with to check for the avialable system models.
release.list_system_models('3.10')

In [ ]:
# Specify the database version and model in variables.
db_version='3.10'
db_system_model='cutoff'
print(f'The background database has been set to ecoinvent-{db_version}-{db_system_model}')

In [ ]:
# Preemptive check, if the desired database is already present.
# If not: Importing database.. This may take some time.
if f'ecoinvent-{db_version}-{db_system_model}' in bd.databases:
    print(f'ecoinvent {db_version} is already present in the project')
else:
    bi.import_ecoinvent_release(
        version=db_version,
        system_model=db_system_model,
        username=os.getenv('USER'),
        password=os.getenv('PASSCODE')
    )

In [ ]:
# Checking project databases
bd.databases

### Premise

In [ ]:
# Create a premise database based on an existing ecoinvent background database
ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
    biosphere_name = "ecoinvent-3.10-biosphere", # name of biosphere database in brightway project if different from "biosphere3"
    key=os.getenv('KEY'), # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
    keep_source_db_uncertainty=False, # False by default, set to True if you want to keep ecoinvent's uncertainty data
    keep_imports_uncertainty=False, # False by default, set to True if you want to keep the uncertainty data of the additional inventories
    use_absolute_efficiency=True, # False by default, set to True if you want to use the IAM's absolute efficiencies
)

In [ ]:
# Add additional IAM projections, which change the future scenario data; for all transformation functions, see https://github.com/polca/premise/blob/master/examples/examples.ipynb
ndb.update() # Leave empty to update all sectors

In [ ]:
# Save newly created database to brightway as a new background database, one for each defined year
ndb.write_db_to_brightway() # Give custom names like ndb.write_db_to_brightway(name=["my_custom_name_1", "my_custom_name_2"])

In [ ]:
bd.databases

### Step 3: Match foreground database

Your foreground database contains the LCI data of your product system. This has to be prepared in an excel file. I included an example file with explainations and hints on how to structure your LCI data in order for the import to function correctly.  

> You can continue with the template to see how it works OR prepare your LCI data for your LCA.  

#### Creating codes for activities

As each activity has to be equipped with an unique code, I included the following hash function. You can insert an activity and use the generated hash as its code in your excel file.

In [ ]:
# Optional: Hash your activities to create individual codes for them.
activity = "<activity_name>"

# Calculate the MD5 hash
code = hashlib.md5(activity.encode()).hexdigest()
print(code)

#### Import foreground database

At this point, you have to decide, which background database you want to use in your LCA. Per default, the declared version of the previous step will be used. If you wish to change that, adjust the variables in Step 2 accordingly.

In [ ]:
# Setting variables for background database.
bg_db = f'ecoinvent-{db_version}'
bg_sys = f'ecoinvent-{db_version}-{db_system_model}'
bg_bio = f'ecoinvent-{db_version}-biosphere'

In [ ]:
# Setting variables for premise background database.
bg_db = f'ei_cutoff_3.10_image_SSP2-RCP19_2050 2025-05-28'
bg_sys = f'ecoinvent-{db_version}-{db_system_model}'
bg_bio = f'ecoinvent-{db_version}-biosphere'

In [ ]:
# Set the foreground database.
excel_file = "example_template.xlsx"
fg_db = bi.ExcelImporter(excel_file)

# Matching activities against themselves and against ecoinvent system model and biosphere to check for inconsistencies.
fg_db.apply_strategies()
fg_db.match_database(fields=["name", "unit", "reference product", "location"])
fg_db.match_database(bg_sys, fields=["name", "unit", "location", "reference product"])
fg_db.match_database(bg_bio, fields=["name", "categories", "location"])

# Checking for unlinked exchanges.
fg_db.statistics()

In [ ]:
# In case any unlinked exchanges are present, locate the missmatches and fix them.
list(fg_db.unlinked)

In [ ]:
# If no unlinked exchanges are present, save the foreground database to your project.
if fg_db.db_name in bd.databases:
    del bd.databases[fg_db.db_name]
    fg_db.write_database()
else:
    fg_db.write_database()

bd.Database(fg_db.db_name).rename(f'{fg_db.db_name}_{bg_db}')

In [ ]:
# Checking project databases
bd.databases

# Deleting database
# del bd.databases["<database_name>"]

### Step 4: Calculation setup

#### Functional Unit

With our data in place, we now have to define the parameters of our LCA. In our excel file, the data is structured in processes and activities. Brightway lets us choose the number of activities (as functional units) and methods for our LCA.  

Tbd.

In [ ]:
# Checking project databases
bd.databases

In [ ]:
# Choose the database for the LCA
wb = bd.Database("UFO LCA_ei_cutoff_3.10_image_SSP2-RCP19_2050 2025-05-28")

# User choice: calculate all activities or specific ones
user_choice = input("Do you want to calculate all activities or specific ones? (type 'all' or 'specific'): ").strip().lower()

if user_choice == 'all':
    # If the user wants all activities, loop through them
    fu = {act["code"]: {wb.get(act["code"]).id: 1} for act in wb}
elif user_choice == 'specific':
    # Print all available activities
    print("Available Activities:")
    for act in wb:
        print(act['name'])
        
    # Ask the user to enter the names of the activities they want
    specific_names = input("Enter the names of the activities you want to include, separated by commas: ").split(',')

    # Create the functional unit based on user input
    fu = {}
    for name in specific_names:
        cleaned_name = name.strip()  # Remove any extra spaces
        # Find the corresponding activity by name
        matching_activities = [act for act in wb if act['name'].strip().lower() == cleaned_name.lower()]
        
        if matching_activities:
            for act in matching_activities:
                fu[act["code"]] = {wb.get(act["code"]).id: 1}
        else:
            print(f"Warning: Activity with name '{cleaned_name}' not found in database.")
else:
    print("Invalid choice. Please type 'all' or 'specific'.")

print("The functional unit has been set accordingly.")

#### LCIA method

Brightway allowes most LCIA methods to be used in the LCA. Available LCIA methods depend on the database version used in the background.  
> Note: Each LCIA method contains various characterization factors or indicators.  
> E.g. climate change, acidification, water use, etc.

For customization purposes, this template lists all available methods from which you can choose.

In [ ]:
# Use a set to store unique second entries
lcia_methods = set()

# Iterate through the data structure and add the second element of each tuple to the set
for method in list(filter(lambda x: bg_db in x[0], bd.methods)):
    lcia_methods.add(method[1])

print(sorted(lcia_methods))

In [ ]:
# Premise
print(sorted(bd.methods.keys()))

In [ ]:
# Premise
ef_methods = list(filter(lambda x: "EF v3.1 no LT" in x[1], list(bd.methods)))
print("We have", len(ef_methods), "EF v3.1 no LT methods we will evaluate.")

config = {
    "impact_categories": ef_methods
}

# Create our final LCA object used for the calculation in the next step.
data_objs = bd.get_multilca_data_objs(functional_units=fu, method_config=config)

In [ ]:
# Choose your LCIA method
lcia_method = "EF v3.1 no LT"

In [ ]:
# Setting the method config and LCA object.
my_methods = list(filter(lambda x: lcia_method in x[1] and x[0].startswith(bg_db), bd.methods))
print("We have", len(my_methods), f"{lcia_method} methods we will evaluate.")

config = {
    "impact_categories": my_methods
}

# Create our final LCA object used for the calculation in the next step.
data_objs = bd.get_multilca_data_objs(functional_units=fu, method_config=config)

### Step 5: Calculation

In [ ]:
# Calculate MultiLCA
lca = bc.MultiLCA(demands=fu, method_config=config, data_objs=data_objs)
lca.lci()
lca.lcia()

# Choose between single or multi LCA
# lca.score
# Format the results
raw_results = lca.scores
def transform_raw_results(raw_results, wb):
    transformed_results = {}
    
    for (key, hash_value), outcome in raw_results.items():
        # Extract the third value from the first part of the key
        category = key[2]  # This corresponds to the third value (e.g., 'climate change', 'human health', etc.)
        
        # Use wb.get() to fetch the activity name directly
        activity_name = wb.get(hash_value)["name"]
        
        # Construct the new key
        new_key = (activity_name, category)
        
        # Add the new key and outcome to the transformed results
        transformed_results[new_key] = outcome
    
    return transformed_results

lca_results = transform_raw_results(raw_results, wb)

In [ ]:
# Display the results in a table
dfresults = pd.DataFrame.from_dict(lca_results, orient='index')
dfresults.index = pd.MultiIndex.from_tuples(dfresults.index, names=('Activities', lcia_method))
dfresults = dfresults.unstack(level=lcia_method)
dfresults.columns = dfresults.columns.droplevel(0)
dfresults

In [ ]:
# Display the results in a table
dfresults = pd.DataFrame.from_dict(lca_results, orient='index')
dfresults.index = pd.MultiIndex.from_tuples(dfresults.index, names=('Activities', lcia_method))
dfresults = dfresults.unstack(level=lcia_method)
dfresults.columns = dfresults.columns.droplevel(0)
dfresults